### Setup

In [13]:
# Main
import numpy as np
import pandas as pd

# Unit root
from quantileADF import QADF
from quantileTAR import QTAR, QTAR_CustomRport
from report import countryReport, reportCountries

# Utilities
import warnings
from statsmodels.tools.sm_exceptions import IterationLimitWarning
warnings.simplefilter('ignore', IterationLimitWarning)
pd.set_option('display.max_rows', 500)

# Setting up Urls
repoUrl = 'https://raw.githubusercontent.com/vlad-yeghiazaryan/QADF/master/data/'
datasetUrl = repoUrl + 'dataset.csv'
broadREERsUrl = repoUrl + 'broadREERs.csv'
narrowREERsUrl = repoUrl + 'narrowREERs.csv'

In [8]:
broadREERs = pd.read_csv(broadREERsUrl, index_col=0, parse_dates=True)
narrowREERs = pd.read_csv(narrowREERsUrl, index_col=0, parse_dates=True)

# Extracting country information
countries = ['Armenia', 'Belarus', 'Kazakhstan', 'Kyrgyz Republic',
             'Moldova', 'Russian Federation', 'Tajikistan', 'Ukraine']

startDate = '2000-01-01'
startDateWithoutTaj = '1995-01-01'

dataNarrow =  narrowREERs[countries].loc[startDate:]
dataNarrow.index.freq = 'MS'

data = broadREERs[countries].loc[startDate:]
data.index.freq = 'MS'

data.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 251 entries, 2000-01-01 to 2020-11-01
Freq: MS
Data columns (total 8 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Armenia             251 non-null    float64
 1   Belarus             251 non-null    float64
 2   Kazakhstan          251 non-null    float64
 3   Kyrgyz Republic     251 non-null    float64
 4   Moldova             251 non-null    float64
 5   Russian Federation  251 non-null    float64
 6   Tajikistan          251 non-null    float64
 7   Ukraine             251 non-null    float64
dtypes: float64(8)
memory usage: 17.6 KB


In [9]:
# renormalize REERs at 2000q1 = 100
data = 100*data/data.iloc[0,:]

# Changing data
dataLogs = data.apply(np.log, axis=1)
dataPtC = dataLogs.diff()[1:]

### Report

In [17]:
dataLogs.head(2)

,Armenia,Belarus,Kazakhstan,Kyrgyz Republic,Moldova,Russian Federation,Tajikistan,Ukraine
Date,,,,,,,,
2000-01-01,4.605170,4.605170,4.605170,4.605170,4.605170,4.605170,4.605170,4.605170
2000-02-01,4.618549,4.503724,4.613256,4.612896,4.602628,4.616694,4.599413,4.620071


In [18]:
levels=[0.01,0.05,0.1]
quantiles = np.arange(0.1, 1, 0.1)
drop = ['δ²', 'Lags', 'ρ₁(OLS)', 'CV1%', 'CV5%', 'CV10%']
qadfParams =  {'model':'ct', 'pmax':12, 'ic':'AIC'}
reps = 10

In [19]:
# Broad EAEU logs
logResults, logReport = reportCountries(dataLogs, QADF, qadfParams, quantiles, 
                                        reps, levels, dropColumns=drop)
logReport.to_excel('Outputs/logReport(C&T).xlsx', float_format="%.3f")

Starting Execution:

Armenia finished in: 12.78s
Belarus finished in: 19.72s
Kazakhstan finished in: 10.49s
Kyrgyz Republic finished in: 9.09s
Moldova finished in: 9.13s
Russian Federation finished in: 12.86s
Tajikistan finished in: 9.69s
Ukraine finished in: 8.55s

Total time spent executing: 1m, 32s


In [20]:
logReport

quantile                                         0.1            0.2  \
Countries          Variable/Quantile                                  
Armenia            α₀(τ)                   0.24(0.2)      0.21(0.1)   
                   ρ₁(τ)                   0.94(0.2)      0.95(0.2)   
                   Half-lives (years)              1              1   
                   tₙ(τ)                 -2.41(0.1)*   -3.2(0.0)***   
                   QKS                     3.2(0.1)*                  
Belarus            α₀(τ)                    0.5(0.1)   0.44(0.0)***   
                   ρ₁(τ)                  0.89(0.1)*    0.9(0.0)***   
                   Half-lives (years)              0              1   
                   tₙ(τ)                 -1.68(0.1)*  -2.88(0.0)***   
                   QKS                  5.54(0.0)***                  
Kazakhstan         α₀(τ)                   0.12(0.2)      0.02(0.4)   
                   ρ₁(τ)                   0.97(0.3)      0.99(0.6)   
                   Half-lives (years)              2             11   
                   tₙ(τ)                   -1.1(0.4)     -0.48(0.7)   
                   QKS                      1.9(0.5)                  
Kyrgyz Republic    α₀(τ)                0.41(0.0)***      0.23(0.1)   
                   ρ₁(τ)                0.91(0.0)***     0.95(0.1)*   
                   Half-lives (years)              1              1   
                   tₙ(τ)               -2.39(0.0)***     -2.35(0.2)   
                   QKS                     2.39(0.6)                  
Moldova            α₀(τ)                   0.27(0.1)   0.21(0.0)***   
                   ρ₁(τ)                  0.94(0.1)*     0.95(0.1)*   
                   Half-lives (years)              1              1   
                   tₙ(τ)                 -2.61(0.1)*     -2.38(0.2)   
                   QKS                     2.61(0.3)                  
Russian Federation α₀(τ)               -0.19(0.0)***  -0.17(0.0)***   
                   ρ₁(τ)                   1.04(1.0)      1.03(1.0)   
                   Half-lives (years)              ∞              ∞   
                   tₙ(τ)                   1.09(1.0)      2.13(1.0)   
                   QKS                   4.8(0.0)***                  
Tajikistan         α₀(τ)                0.66(0.0)***   0.35(0.0)***   
                   ρ₁(τ)                0.85(0.0)***   0.92(0.0)***   
                   Half-lives (years)              0              1   
                   tₙ(τ)               -3.41(0.0)***  -2.62(0.0)***   
                   QKS                    3.41(0.1)*                  
Ukraine            α₀(τ)                  -0.03(0.2)      0.08(0.1)   
                   ρ₁(τ)                    1.0(0.8)      0.98(0.2)   
                   Half-lives (years)              ∞              3   
                   tₙ(τ)                   0.06(0.8)     -0.86(0.4)   
                   QKS                  3.88(0.0)***                  

quantile                                         0.3            0.4  \
Countries          Variable/Quantile                                  
Armenia            α₀(τ)                   0.09(0.4)      0.04(0.3)   
                   ρ₁(τ)                   0.98(0.4)      0.99(0.7)   
                   Half-lives (years)              3              6   
                   tₙ(τ)                  -1.32(0.8)     -0.59(0.8)   
                   QKS                                                
Belarus            α₀(τ)                0.47(0.0)***   0.41(0.0)***   
                   ρ₁(τ)                 0.9(0.0)***   0.91(0.0)***   
                   Half-lives (years)              1              1   
                   tₙ(τ)               -4.45(0.0)***  -5.26(0.0)***   
                   QKS                                                
Kazakhstan         α₀(τ)                   0.04(0.3)      0.05(0.4)   
                   ρ₁(τ)                   0.99(0.5)      0.99(0.4)   
                   Half

In [21]:
EAEU = pd.Series((dataLogs.index > '2014-12-01').astype(int), index=dataLogs.index, name='EAEU')
levels=[0.01,0.05,0.1]
quantiles = np.arange(0.1, 1, 0.1)
drop = ['δ²', 'Lags', 'CV1%', 'CV5%', 'CV10%']
qtarParams =  {'threshold_dummy':EAEU, 'model':'ct', 'pmax':12, 'ic':'AIC'}
reps = 10

In [22]:
QTAR_logResults, QTAR_logReport = reportCountries(dataLogs, QTAR, qtarParams, quantiles, reps, levels, 
                                                  customReport=QTAR_CustomRport, dropColumns=drop)

Starting Execution:

Armenia finished in: 14.17s
Belarus finished in: 17.9s
Kazakhstan finished in: 10.28s
Kyrgyz Republic finished in: 10.23s
Moldova finished in: 9.99s
Russian Federation finished in: 12.72s
Tajikistan finished in: 11.18s
Ukraine finished in: 9.59s

Total time spent executing: 1m, 36s


In [23]:
QTAR_logReport

quantile                                           0.1              0.2  \
Countries          Variable/Quantile                                      
Armenia            α₀(τ)                     0.26(0.2)        0.22(0.3)   
                   pre-ρ₁(τ)                 0.94(0.2)        0.95(0.4)   
                   post-ρ₂(τ)                0.94(0.2)        0.95(0.3)   
                   pre-ρ₁(OLS)               0.97(0.4)        0.97(0.4)   
                   post-ρ₂(OLS)              0.96(0.4)        0.96(0.4)   
                   ρ₁(τ)-ρ₂(τ)             0.0001(0.6)      0.0006(0.7)   
                   Half-lives (years)                1                1   
                   pre-tₙ(τ)               -37.02(0.2)      -50.78(0.2)   
                   post-tₙ(τ)              -35.93(0.2)      -49.81(0.2)   
                   pre-QKS                  50.78(0.5)                    
                   post-QKS                 49.81(0.5)                    
Belarus            α₀(τ)                     0.45(0.3)        0.51(0.2)   
                   pre-ρ₁(τ)                  0.9(0.3)        0.89(0.2)   
                   post-ρ₂(τ)                 0.9(0.3)        0.88(0.2)   
                   pre-ρ₁(OLS)                0.9(0.3)         0.9(0.3)   
                   post-ρ₂(OLS)               0.9(0.3)         0.9(0.3)   
                   ρ₁(τ)-ρ₂(τ)             0.0014(0.3)      0.0032(0.9)   
                   Half-lives (years)                1                0   
                   pre-tₙ(τ)               -22.45(0.4)      -68.83(0.2)   
                   post-tₙ(τ)              -22.72(0.4)      -70.61(0.2)   
                   pre-QKS                99.6(0.0)***                    
                   post-QKS             101.14(0.0)***                    
Kazakhstan         α₀(τ)                  0.33(0.0)***     0.31(0.0)***   
                   pre-ρ₁(τ)              0.92(0.0)***     0.93(0.0)***   
                   post-ρ₂(τ)             0.92(0.0)***     0.92(0.0)***   
                   pre-ρ₁(OLS)               0.94(0.2)        0.94(0.2)   
                   post-ρ₂(OLS)             0.93(0.1)*       0.93(0.1)*   
                   ρ₁(τ)-ρ₂(τ)             0.0086(1.0)       0.007(1.0)   
                   Half-lives (years)                1                1   
                   pre-tₙ(τ)              -17.61(0.1)*   -73.46(0.0)***   
                   post-tₙ(τ)            -18.4(0.0)***   -75.87(0.0)***   
                   pre-QKS               81.34(0.0)***                    
                   post-QKS              83.83(0.0)***                    
Kyrgyz Republic    α₀(τ)                  0.47(0.0)***     0.34(0.0)***   
                   pre-ρ₁(τ)              0.89(0.0)***     0.92(0.0)***   
                   post-ρ₂(τ)             0.89(0.0)***     0.92(0.0)***   
                   pre-ρ₁(OLS)               0.95(0.4)        0.95(0.4)   
                   post-ρ₂(OLS)              0.95(0.3)        0.95(0.3)   
                   ρ₁(τ)-ρ₂(τ)             0.0029(0.9)      0.0018(1.0)   
                   Half-lives (years)                1                1   
                   pre-tₙ(τ)              -66.37(0.1)*   -78.98(0.0)***   
                   post-tₙ(τ)             -66.68(0.1)*   -79.07(0.0)***   
                   pre-QKS                  83.17(0.2)                    
                   post-QKS                 84.51(0.2)                    
Moldova            α₀(τ)                     0.29(0.1)     0.32(0.0)***   
                   pre-ρ₁(τ)                 0.93(0.2)     0.93(0.0)***   
                   post-ρ₂(τ)               0.93(0.1)*     0.93(0.0)***   
                   pre-ρ₁(OLS)               0.96(0.4)        0.96(0.4)   
                   post-ρ₂(OLS)              0.96(0.3)        0.96(0.3)   
                   ρ₁(τ)-ρ₂(τ)             0.0025(0.8)      0.0028(1.0)   
                   Half-lives (years)                1                1   
                   pre-tₙ